# Chat With LLM

## Prepare

In [73]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.llms import Tongyi
from pydantic import BaseModel

In [74]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [75]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(trace_name="chat_with_llm", user_id="langchain_book")

## LLM

In [76]:
parser = StrOutputParser()

In [9]:
# import nest_asyncio
# nest_asyncio.apply()

### GPT35

In [59]:
async def gpt35(input: str, temp = 0.3):
    prompt = ChatPromptTemplate.from_template("""{question}""")
    llm = ChatOpenAI(model = "gpt-3.5-turbo-1106", streaming = True, temperature = temp)
    chain = prompt | llm | parser
    result = chain.stream({"question": input}, config = {"callbacks": [handler]})
    for r in result:
        print(r, end = "")

In [60]:
await gpt35("你是什么模型")

我是一个基于人工智能的语言模型，可以进行对话交流并回答问题。我并不是一个具体的物理模型或数学模型。

### GPT4

In [46]:
async def gpt4(input: str, temp = 0.3):
    prompt = ChatPromptTemplate.from_template("""{question}""")
    llm = ChatOpenAI(model = "gpt-4-1106-preview", streaming = True, temperature = temp)
    chain = prompt | llm | parser
    result = chain.stream({"question": input}, config={"callbacks": [handler]})
    for r in result:
        print(r, end = "")

In [47]:
await gpt4("你是什么模型")

我是由 OpenAI 开发的大型语言模型，名为 ChatGPT。我的架构基于 GPT（生成预训练变换器）系列模型，这是一种深度学习模型，专门设计用于理解和生成自然语言文本。我的训练涉及大量的文本数据，使我能够回答问题、撰写文章、翻译语言、进行对话等。不过，请注意，我的知识是有截止日期的，我不会了解在我训练数据截止日期之后发生的事件或发展。

### 通义千问

In [122]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Tongyi

async def tongyi(input: str, temp = 0.3):
    prompt = PromptTemplate(template="""{question}""", input_variables=["question"])
    llm = Tongyi(temperature = temp)
    chain = LLMChain(prompt = prompt, llm = llm)
    result = chain.stream(input)
    for r in result:
        print(r, end = "")

In [123]:
await tongyi("你是什么模型")

{'question': '你是什么模型', 'text': '我是来自阿里云的大规模语言模型，我叫通义千问。'}

### ChatGLM

In [119]:
async def chatglm(input: str, temp = 0.3):
    prompt = PromptTemplate(template="""{question}""", input_variables=["question"])
    llm = Tongyi(model_name = "chatglm3-6b")
    chain = LLMChain(prompt = prompt, llm = llm)
    result = chain.stream({"question": input})
    for r in result:
        print(r, end = "")

In [120]:
await chatglm("你是什么模型")

{'question': '你是什么模型', 'text': '\n 我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。'}

In [124]:
await tongyi("""
请帮我翻译成中文：Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on “distance”. But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.""")

{'question': '\n请帮我翻译成中文：Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on “distance”. But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.\n\nThe MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.', 'text': '基于距离的矢量数据库检索将查询在高维空间中嵌入（表示），并通过“距离”

In [125]:
await gpt4("我对RAG的实现原理有一点疑惑：我们通常将作为答案的背景知识文本编码到向量数据库，然后通过提问内容的向量编码查询，但问题和答案之间的向量相似性真的可以保障吗？")

RAG（Retrieval-Augmented Generation）模型是一种结合了检索（Retrieval）和生成（Generation）的机器学习模型，通常用于自然语言处理任务，如问答系统、文本摘要等。RAG模型的核心思想是通过检索相关的文档或知识片段，然后将这些信息融入到生成模型中，以此来生成更加准确和丰富的答案。

在RAG模型中，确实存在将问题和答案的文本编码为向量，并存储在向量数据库中。当有一个新的问题时，模型会将这个问题编码为一个向量，并在向量数据库中寻找与之相似的向量，这些相似的向量通常对应于潜在的答案或相关信息。然后，这些检索到的信息会被用作生成模型的一部分，辅助生成答案。

至于问题和答案之间的向量相似性是否可以保障，这实际上取决于几个因素：

1. 向量表示的质量：如果使用的是高质量的预训练语言模型（如BERT、RoBERTa等）来编码文本，那么这些模型通常能够捕捉到文本的深层语义信息，从而生成更加语义相关的向量表示。

2. 检索算法的效果：检索算法需要能够有效地从向量数据库中找到与问题最相关的文档或知识片段。这通常涉及到一些相似度计算方法，如余弦相似度、欧氏距离等。

3. 训练数据和目标任务的匹配度：如果训练数据与目标任务高度相关，那么模型在训练过程中学到的知识将更有可能在实际应用中找到与问题相关的答案。

4. 模型的微调和优化：通过对特定任务的微调，可以进一步提高模型对于问题和答案之间相似性的捕捉能力。

尽管有这些因素的影响，RAG模型仍然可能面临挑战，因为问题和答案之间的相似性并不总是直接的或者显而易见的。有时候，问题可能需要推理或者结合多个知识点才能找到答案，这就需要模型具有更高级的理解和推理能力。因此，研究人员和工程师通常会继续探索更好的编码方法、检索技术和模型架构，以提高RAG模型在实际应用中的性能。

In [126]:
await gpt4("在RAG的最佳实践中，是否应当尽量获得高质量的问答对文本，并将其编码到向量数据库，然后将用户的问题与库中的问题做比较，而不是与库中的答案做比较，这样可以有效提高答案的命中率？")

在实现基于检索的生成模型（Retrieval-Augmented Generation, RAG）时，确实应该关注高质量问答对的获取和使用。RAG模型结合了检索（retrieval）和生成（generation）两个步骤，以改善生成的答案质量。以下是一些最佳实践：

1. **高质量问答对**: 确保问答对的质量是至关重要的。高质量的问答对可以提供更准确、更相关的信息，有助于生成模型产生更好的答案。这些问答对应该涵盖广泛的主题，并且答案应该是准确和信息丰富的。

2. **问题编码与匹配**: 在RAG模型中，用户的问题通常与问答对中的问题进行比较，而不是直接与答案比较。这是因为用户提出的问题和数据库中存储的问题在语义上更容易匹配。一旦找到最匹配的问题，相应的答案就可以用来辅助生成模型产生答案。

3. **向量数据库**: 使用高效的向量搜索技术来存储和检索问题的向量表示。这通常涉及到使用像FAISS这样的库来加速相似性搜索。问题的向量表示应该能够捕获语义信息，以便在检索阶段能够找到最相关的问答对。

4. **上下文编码**: 在编码问题时，考虑到问题的上下文可以提高检索的准确性。这意味着不仅仅是问题本身，相关的上下文信息（如前后文或附加的背景信息）也应该被编码进向量中。

5. **连续学习**: 随着时间的推移，问答库应该不断更新和扩展，以包括新的信息和数据。此外，可以通过持续学习（continual learning）来微调检索和生成模型，以保持其性能。

6. **多模态数据**: 如果可能，考虑使用多模态数据（如文本、图像、表格等）来丰富问答对，这样可以提供更全面的信息，有助于生成更准确的答案。

7. **用户反馈**: 利用用户反馈来评估和改进模型。用户对生成答案的满意度可以作为一个重要的指标，指导模型的迭代和优化。

8. **评估和测试**: 定期对模型进行全面的评估和测试，以确保其性能满足预期。使用标准化的评估指标和测试集可以帮助监控模型的进展。

总之，RAG模型的最佳实践应该包括获取和使用高质量问答对、有效的问题编码与匹配、持续的模型优化和更新，以及定期的评估和用户反馈。这样可以确保模型能够提供高质量和相关性强的答案。